## Gray-Scott Model

Represents the reaction and diffusion of two generic chemical species U and V.  The concentration of each in space is represented by u and v.

* Each chemical diffuses through space at its own rate
* U is added at a const feed rate into the system
* Two units of species V can turn a unit of U into V: $$2V + U \rightarrow 3V$$
* Const kill rate removing V

Represented by PDE for the concentrations of $u(x, y, t)$ and $v(x, y, t)$:

$$\frac{\partial u}{\partial t} = D_u \nabla ^2 u - uv^2 + F(1 - u)$$
$$\frac{\partial v}{\partial t} = D_v \nabla ^2 v + uv^2 - (F + k)v$$